## Testing and validation of ERiC implementation

In this notebook, we test the outputs of both ELKI ERiC and our implementation of ERiC by comparing the outputs of the sample datasets from the two artificial datasets given at: https://elki-project.github.io/datasets/

In [1]:
import numpy as np
import pandas as pd

from lib import *

from elki_eric import elki_eric
import elki_parser
import validation

Here we define methods that load the data and run both algorithms. In the *validation.py* file we compare the sizes and content of the clusters. We also output the structure so that it can be checked manually.

In [7]:
def load_dataset(file):
    with open(file) as f:
        lines = [line.rstrip() for line in f if "#" not in line]
        lines = [",".join(line.split(" ")) for line in lines]
    
    df = pd.DataFrame([sub.split(",") for sub in lines], columns=["x1","x2","label"])
    X = df[["x1","x2"]]
    X = X.astype(float)
    
    y = df["label"]
    return X, y
    
    
def run_ERiC(df, k=10, min_samples=2, delta_affine=0.5, delta_dist=0.5):
    D = df.to_numpy(dtype=np.float64)
    point_info, partitions = make_partitions(D, k)

    models, clusters = cluster_partitions(D, partitions, point_info, delta_affine, delta_dist, min_samples)

    cluster_info = compute_cluster_list(clusters, D)
    cluster_info = build_hierarchy(cluster_info, delta_affine, delta_dist, D.shape[1])
    
    return cluster_info    
    
    
def run_ELKI_ERiC(df, k=10, min_samples=2, delta_affine=0.5, delta_dist=0.5, output_file=None):
    D = df.to_numpy(dtype=np.float64)
    
    elki_eric(
        X, 
        k=k, 
        dbscan_minpts=min_samples, 
        alpha=0.85, 
        delta_dist=delta_dist, 
        delta_affine=delta_affine, 
        output_file_name=output_file)
    
    df1_output = elki_parser.read_file(output_file)
    cluster_info = elki_parser.parse_file(df1_output)
    
    return cluster_info   

### Mouse dataset

In [8]:
X, y = load_dataset("sample_dataset/mouse.csv")

# Our implementation
cluster_info_eric = run_ERiC(X)
# ELKI implementation
elki_output_df1 = "elki_df1_output.txt"
cluster_info_elki_eric = run_ELKI_ERiC(X, output_file=elki_output_df1)

# Run validation
validation.validate(cluster_info_eric, cluster_info_elki_eric)

1
2
***
Run elki
Saving ELKI results in elki_df1_output.txt
Writing completed.
The implementations return the same number of clusters.
No. of clusters (our ERiC): 2
No. of clusters (ELKI ERiC): 2

The implementations return the same number of lambdas.
No. of lambdas (our ERiC): {1: 1, 2: 1}
No. of lambdas (ELKI ERiC): {1: 1, 2: 1}

Our ERiC structure:
Partition  1
--- cluster 0  size: 24
------ points
------ parents
[2]
Partition  2
--- cluster 0  size: 476
------ points
------ parents
[]

ELKI ERic structure
Partition  1
--- cluster 0  size: 24
------ points
------ parents
[2]
Partition  2
--- cluster 0  size: 476
------ points
------ parents
[]

Cluster sizes were identical for lambda=1
Cluster values are identical for lambda=1
Cluster sizes were identical for lambda=2
Cluster values are identical for lambda=2

Validation result: The outputs of the algorithms are identical.


### Vary density dataset

In [9]:
X, y = load_dataset("sample_dataset/vary_density.csv")

# Our implementation
cluster_info_eric = run_ERiC(X)
# ELKI implementation
elki_output_df2 = "elki_df2_output.txt"
cluster_info_elki_eric = run_ELKI_ERiC(X, output_file=elki_output_df2)

# Run validation
validation.validate(cluster_info_eric, cluster_info_elki_eric)

1
2
***
Run elki
Saving ELKI results in elki_df2_output.txt
Writing completed.
The implementations return the same number of clusters.
No. of clusters (our ERiC): 2
No. of clusters (ELKI ERiC): 2

The implementations return the same number of lambdas.
No. of lambdas (our ERiC): {1: 1, 2: 1}
No. of lambdas (ELKI ERiC): {1: 1, 2: 1}

Our ERiC structure:
Partition  1
--- cluster 0  size: 3
------ points
------ parents
[2]
Partition  2
--- cluster 0  size: 147
------ points
------ parents
[]

ELKI ERic structure
Partition  1
--- cluster 0  size: 3
------ points
------ parents
[2]
Partition  2
--- cluster 0  size: 147
------ points
------ parents
[]

Cluster sizes were identical for lambda=1
Cluster values are identical for lambda=1
Cluster sizes were identical for lambda=2
Cluster values are identical for lambda=2

Validation result: The outputs of the algorithms are identical.


As seen, the outputs for both sample datasets are identical.